# Vectorize

Only to write shape of count word and tfidf

In [1]:
import pickle
import pandas as pd
#Loading from last EDA notebook
data = pd.read_pickle('eda.pkl')
data.shape

(68762, 37)

In [2]:
data.head(2)

,appln,appln_id,family_id,sets,direct,granted,earliest_appln,earliest_filing_date,publn_date,publn,...,citing_appln_id,citing_family_id,num_citing_family,earliest_applt,current_applt,appln_title,appln_abstr,assignee,application_date,year
0,WO9105114W,47158713,1239277,2,1,1,US55592190A,1990-07-19,1993-02-18,WO9303532A1,...,45621927,24395918,30,LUZ ELECTRIC FUEL ISRAEL LTD.,LUZ ELECTRIC FUEL ISRAEL LTD.,ELECTRICAL ENERGY SYSTEM,An electrical energy system including an elect...,luz electric fuel israel ltd,1990-07-19,1990
1,US55592190A,51797454,1239277,2,1,1,US55592190A,1990-07-19,1992-06-09,US5121044A,...,52393850\n16916549\n54353461\n48212422\n457802...,27071024\n1239277\n25508001\n22545199\n2636904...,30,"Luz Electric Fuel Israel, Ltd.","Luz Electric Fuel Israel, Ltd.",Electrical energy system,An electrical energy system including an elect...,luz electric fuel israel ltd,1990-07-19,1990


**saving file as we get index error in cleaning process**

In [3]:
data.to_csv('eda.csv')

In [4]:
df = pd.read_csv('eda.csv')
df=df.rename(columns={'Unnamed: 0':'random_columns'})
df.shape

(68762, 38)

In [5]:
df.head(2)

,random_columns,appln,appln_id,family_id,sets,direct,granted,earliest_appln,earliest_filing_date,publn_date,...,citing_appln_id,citing_family_id,num_citing_family,earliest_applt,current_applt,appln_title,appln_abstr,assignee,application_date,year
0,0,WO9105114W,47158713,1239277,2,1,1,US55592190A,1990-07-19,1993-02-18,...,45621927,24395918,30,LUZ ELECTRIC FUEL ISRAEL LTD.,LUZ ELECTRIC FUEL ISRAEL LTD.,ELECTRICAL ENERGY SYSTEM,An electrical energy system including an elect...,luz electric fuel israel ltd,1990-07-19,1990
1,1,US55592190A,51797454,1239277,2,1,1,US55592190A,1990-07-19,1992-06-09,...,52393850\n16916549\n54353461\n48212422\n457802...,27071024\n1239277\n25508001\n22545199\n2636904...,30,"Luz Electric Fuel Israel, Ltd.","Luz Electric Fuel Israel, Ltd.",Electrical energy system,An electrical energy system including an elect...,luz electric fuel israel ltd,1990-07-19,1990


In [6]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
lemm = WordNetLemmatizer()
nltk.download('wordnet')
clean_text = [] # defining corpus
for i in range(0, len(df['appln_abstr'])): # giving range of values from 0 to 68762
    processed_text = re.sub(r'<[^>]+>', ' ', df['appln_abstr'][i]) # using i for all values instead of 1 column
    processed_text = processed_text.lower()
    processed_text = processed_text.split()
    lemm = WordNetLemmatizer()
    stopword_set = set(stopwords.words('english'))
    processed_text = [lemm.lemmatize(word) for word in processed_text if not word in stopword_set]
    processed_text = ' '.join(processed_text )
    clean_text.append(processed_text) # finally attach all these cleaned values to corpus directory

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hassan.sherwani\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:
# convert to series to check words
text_corpus = pd.Series(clean_text)
df['clean'] = text_corpus
df.head(2)

,random_columns,appln,appln_id,family_id,sets,direct,granted,earliest_appln,earliest_filing_date,publn_date,...,citing_family_id,num_citing_family,earliest_applt,current_applt,appln_title,appln_abstr,assignee,application_date,year,clean
0,0,WO9105114W,47158713,1239277,2,1,1,US55592190A,1990-07-19,1993-02-18,...,24395918,30,LUZ ELECTRIC FUEL ISRAEL LTD.,LUZ ELECTRIC FUEL ISRAEL LTD.,ELECTRICAL ENERGY SYSTEM,An electrical energy system including an elect...,luz electric fuel israel ltd,1990-07-19,1990,electrical energy system including electric ut...
1,1,US55592190A,51797454,1239277,2,1,1,US55592190A,1990-07-19,1992-06-09,...,27071024\n1239277\n25508001\n22545199\n2636904...,30,"Luz Electric Fuel Israel, Ltd.","Luz Electric Fuel Israel, Ltd.",Electrical energy system,An electrical energy system including an elect...,luz electric fuel israel ltd,1990-07-19,1990,electrical energy system including electric ut...


### Bag Of Words

We screened out the terms that did not appear in more than
95% of abstracts to exclude terms that rarely appeared. With the remaining 238 terms, consequently,
the 3724 × 238 TF-IDF matrix was obtained.

In [8]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
import gensim
CountVectorizer()

CountVectorizer()

In [9]:
bow_vectorizer = CountVectorizer(max_df=0.95, min_df=5, stop_words="english",analyzer = 'word')
# bag-of-words feature matrix
bow = bow_vectorizer.fit_transform(df['clean'])
bow.shape

(68762, 12630)

### TFIDF

In [10]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=5, stop_words="english",analyzer = 'word')
# TF-IDF feature matrix
tfidf = tfidf_vectorizer.fit_transform(df['clean'])
tfidf.shape

(68762, 12630)